# Data Wrangling

In [3]:
import pandas as pd
import sys
import os

# Agrega el path al directorio raíz del proyecto
sys.path.append(os.path.abspath(".."))

In [4]:
# Extraer datos de un archivo CSV
df_eventos = pd.read_csv('../data/raw/eventos.csv')
df_normal = pd.read_csv('../data/raw/normal.csv')

## Nulos

In [5]:
# Elimina todos los registros donde Start_Frame es nulo
df_eventos = df_eventos[df_eventos['Start_Frame'].notnull()]

In [6]:
# Rellena los nulos con 0
df_eventos = df_eventos.fillna(0)

In [7]:
df_eventos.shape

(340, 5)

## Duplicados

In [8]:
# Elimina el Video Robbery138_x264.mp4
df_eventos = df_eventos[df_eventos['Video'] != 'Robbery138_x264.mp4']

In [9]:
df_eventos.shape

(339, 5)

## Outliers

In [10]:
import cv2
# Crea una función para ver la duración de cada video 
def duracion_video(video, evento):
    video_path = f"../data/Anomaly-Detection-Dataset/Anomaly-Videos/{evento}/{video}"
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duracion = frame_count/fps
    return duracion

In [11]:
# df_eventos
df_eventos['duracion'] = df_eventos.apply(lambda x: duracion_video(x['Video'], x['Evento']), axis=1)

OpenCV: Couldn't read video stream from file "../data/Anomaly-Detection-Dataset/Anomaly-Videos/Abuse/Abuse001_x264.mp4"


ZeroDivisionError: float division by zero

In [ ]:
# df_normal
df_normal['duracion'] = df_normal.apply(lambda x: duracion_video(x['Video'], 'Normal'), axis=1)

In [2]:
# Boxplot de la duración de los videos
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(x='Evento', y='duracion', data=df_eventos)
plt.title('Duración de los videos')
plt.show()

NameError: name 'df_eventos' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Evento', y='duracion', data=df_normal)
plt.title('Duración de los videos')
plt.show()

In [ ]:
# Elimina los videos con duración menor a 10 segundos
df_eventos = df_eventos[df_eventos['duracion'] > 10]
df_normal = df_normal[df_normal['duracion'] > 10]

# Elimina los videos con duración mayor a 60 segundos
df_eventos = df_eventos[df_eventos['duracion'] < 60]
df_normal = df_normal[df_normal['duracion'] < 60]

In [ ]:

plt.figure(figsize=(10, 6))
sns.boxplot(x='Evento', y='duracion', data=df_eventos)
plt.title('Duración de los videos')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Evento', y='duracion', data=df_normal)
plt.title('Duración de los videos')
plt.show()

## Unión

In [18]:
# Une los dos dataframes
df = pd.concat([df_eventos, df_normal])

In [20]:
df[df['Evento'] == 'Normal'].shape

(150, 5)

In [21]:
df.to_csv('../data/clean/data.csv', index=False)